<a href="https://colab.research.google.com/github/hussain0048/Projects-/blob/master/Image_Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Introduction**

In the realm of machine learning and computer vision, the quality of your model's output heavily relies on the quality of the input data. Image data preprocessing is a crucial step that can significantly influence the performance of your model.
When given a dataset, the preprocessing can have various steps depending on
a) what type of data you're looking at (text, images, time series, ...)
b) what models you want to train

This blog will walk you through the essentials of image data preprocessing in Python, using popular libraries like OpenCV, PIL, and TensorFlow.

# **Table of Content**



1.   What is image Preprocessing
2.   Tools and Libraries
3.   Image Preprocessing steps



# **What is image preprocessing?**

**What is preprocessing?**

Preprocessing describes the process of cleaning and converting a 'raw' (i.e. unprocessed) dataset into a clean dataset.

**Why Image Data Preprocessing?**

Before feeding images into a machine learning model, preprocessing is necessary for several reasons:

1- Normalization: Ensures that pixel values are within a specific range.

2- Resizing: Standardizes the input size for uniformity.

3- Augmentation: Increases the diversity of the training data without actually collecting new data.

4- Noise Reduction: Removes unwanted artifacts that can distort the image.

# T**ools and Libraries**

Several libraries exist that make it easier to preprocess images. For example, you can use **scikit-image**, **OpenCV **or **Pillow**. Each library has different functionalities, pros and cons. In this notebook we will stick to scikit-image.

**Tools and Libraries**

Python offers several libraries to handle image data preprocessing:

1- **OpenCV:** A powerful library for computer vision tasks.

2- **PIL (Pillow):** A Python Imaging Library that adds image processing capabilities.

3- **TensorFlow:** An end-to-end open-source platform for machine learning that includes preprocessing utilities.

4-  **scikit-image** scikit-image is a collection of algorithms for image processing.

Let's dive into some common preprocessing steps using these libraries.

# **Image Data Preprocessing Steps**

As mentioned already, the preprocessing steps you will need for your dataset depend on the nature of the dataset and models you want to train. Possible preprocessing steps for images are:

## **Data Loading**

Getting images ready for use means taking them from wherever they're stored and bringing them into memory. You can do this with tools like PIL or OpenCV. This makes the images easier to work with and study.

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import glob
import os
import random
import matplotlib
import warnings

import numpy as np
import matplotlib.pyplot as plt

from skimage import io
from skimage import img_as_float
from skimage.transform import resize, rotate
from skimage.color import rgb2gray

%matplotlib inline
warnings.simplefilter('ignore')

In [ ]:
# Create a list of all images
root_path = os.path.expanduser('~/ml_basics/images/')
all_images = glob.glob(root_path + '/*.jpg')
# To avoid memory errors we will only use a subset of the images
all_images = random.sample(all_images, 500)

# **References**

[1- Image Preprocessing](https://github.com/zotroneneis/machine_learning_basics/blob/master/image_preprocessing.ipynb)

[2-Image Data Preprocessing Techniques You Should Know](https://thecleverprogrammer.com/2024/06/05/image-data-preprocessing-techniques-you-should-know/?fbclid=IwZXh0bgNhZW0CMTEAAR2Qfq3em9lQDc4msZcLMmimg9nPX-RvBKMZPVNg40Tn_q5e1UK9kHfOl18_aem_ZmFrZWR1bW15MTZieXRlcw)